In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#pip install git+https://github.com/huggingface/transformers

In [3]:

!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 49.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.29.165 which is incompatible.


In [4]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install textblob
!pip install mlxtend
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.0 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-se0urj5i
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-se0urj5i
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 96bf0

# Data Loading and Preprocessing

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import preprocess_kgptalkie as ps

df = pd.read_csv("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/IMDB-Dataset.csv")
df = df.sample(10_000)
df.head()


,review,sentiment
33437,"Three young college women, Simona (Iva Krajnc)...",negative
41108,Was this supposed to be a comedy? The black ca...,negative
40839,"This movie is about basically human relations,...",positive
47203,"The story idea is excellent. Unfortunately, th...",negative
32269,"I'd heard this Japanese flick is edgy, creativ...",negative


In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
# preprocessing
df['word_counts'] = df['review'].apply(lambda x: ps.get_wordcounts(x))
df['char_counts'] = df['review'].apply(lambda x: ps.get_charcounts(x))
df['avg_wordlength'] = df['review'].apply(lambda x: ps.get_avg_wordlength(x))
df['stopwords_counts'] = df['review'].apply(lambda x: ps.get_stopwords_counts(x))

In [8]:
df.head()

,review,sentiment,word_counts,char_counts,avg_wordlength,stopwords_counts
33437,"Three young college women, Simona (Iva Krajnc)...",negative,284,1440,5.070423,135
41108,Was this supposed to be a comedy? The black ca...,negative,263,1175,4.467681,123
40839,"This movie is about basically human relations,...",positive,161,732,4.546584,87
47203,"The story idea is excellent. Unfortunately, th...",negative,147,740,5.034014,75
32269,"I'd heard this Japanese flick is edgy, creativ...",negative,102,535,5.245098,40


In [9]:
df['review'] = df['review'].str.lower()
df['review'] = df['review'].apply(lambda x : ps.remove_html_tags(x))

/opt/conda/lib/python3.10/site-packages/preprocess_kgptalkie/utils.py:97: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(x, 'lxml').get_text().strip()


In [10]:
df.head()

,review,sentiment,word_counts,char_counts,avg_wordlength,stopwords_counts
33437,"three young college women, simona (iva krajnc)...",negative,284,1440,5.070423,135
41108,was this supposed to be a comedy? the black ca...,negative,263,1175,4.467681,123
40839,"this movie is about basically human relations,...",positive,161,732,4.546584,87
47203,"the story idea is excellent. unfortunately, th...",negative,147,740,5.034014,75
32269,"i'd heard this japanese flick is edgy, creativ...",negative,102,535,5.245098,40


In [11]:
df['word_counts'].value_counts()

word_counts
124    84
123    80
126    79
127    76
130    75
       ..
904     1
739     1
684     1
921     1
966     1
Name: count, Length: 847, dtype: int64

# Data Preparation for ML
1. Custom Dataset
2. Evaluation Metrics
3. Training Arguments
4. Trainer
5. Training
6. Testing

In [12]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split


In [13]:
class CustomDataset(Dataset):
    def __init__(self,texts,labels,tokenizer,max_len = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = str(self.texts[idx])
        label = torch.tensor(self.labels[idx])
        
        encoding = self.tokenizer(text,truncation=True,padding = "max_length",max_length = self.max_len)
        
        return {
            'input_ids':encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'labels':label
            
        }
        

In [14]:
# import torch
# print(f"CUDA available: {torch.cuda.is_available()}")
# print(f"Device: {next(model.parameters()).device}")

In [15]:
# pip install torch --extra-index-url https://download.pytorch.org/whl/cu117

In [16]:
#prepare tokenizer and model

from transformers import AutoTokenizer, AutoModelForSequenceClassification


checkpoint = 'distilbert-base-uncased'
#device = "cuda"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 2).to(device)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# converting our dataframe to our CustomDataset

In [17]:
X = df['review'].tolist()

label2id = {'psotive':1,'negative':0}
id2label = {1:'positive',0:'negative'}

y = df['sentiment'].map(label2id).tolist()

dataset = CustomDataset(X,y,tokenizer)

In [18]:
train_dataset,test_dataset = train_test_split(dataset,test_size=0.2,random_state = 42)

In [19]:
#confu
from sklearn.metrics import accuracy_score,f1_score
def compute_metrics(example):
    true_labels = example.label_ids
    preds = example.predictions.argmax(-1)
    
    f1 = f1_score(true_labels,preds, average = 'weighted')
    accuracy = accuracy(true_labels,preds)
    
    return {'accuracy':accuracy,
            'f1_score':f1}
    

In [ ]:
from botocore import compress

In [20]:
from transformers import TrainingArguments, Trainer


2024-04-20 16:19:26.499617: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 16:19:26.499711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 16:19:26.660455: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
No module named 'botocore.compress'

In [ ]:


batch_size = 16
model_name = "distilbert_finetuned_setiment"

args = TrainingArguments(
    output_dir = "/kaggle/working/output",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size = batch_size,
    learning_rate = 2e-5,
    num_train_epochs = 1,
    evaluation_strategy = 'epoch'
)
     

In [ ]:
trainer = Trainer(model = model,
                  args = args,
                  train_datasets = train_datasets,
                  eval_datasets = test_datasets,
                  compute_metrics = compute_metrics,
                  tokenizer = tokenizer
                 )

In [ ]:
trainer.train()

In [ ]:

trainer.save_model(model_name)

# Model Testing

In [ ]:
text = "i love this product"
pipe = pipeline('text-classification', model_name)
pipe(text)

In [ ]:
id2label

In [ ]:
tok = AutoTokenizer.from_pretrained(model_name)
mod = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
def get_prediction(text):
  input_ids = tok.encode(text, return_tensors='pt')
  output = mod(input_ids)

  preds = torch.nn.functional.softmax(output.logits, dim=-1)

  prob = torch.max(preds).item()

  idx = torch.argmax(preds).item()
  sentiment = id2label[idx]

  return {'sentiment':sentiment, 'prob':prob}
     

In [ ]:
text = "i love this product"
get_prediction(text)